In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline


In [ ]:
import pickle
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


from dataloader.builder import build_dataset
from model.mlp import MLP
from model.ensemble import MLPEnsemble
from uncertainty_estimator.masks import build_masks, DEFAULT_MASKS
from experiment_setup import build_estimator
from analysis.metrics import get_uq_metrics
from experiments.utils.data import scale, split_ood, multiple_kfold
import torch

plt.rcParams['figure.facecolor'] = 'white'
torch.cuda.set_device(1)

In [ ]:
config = {
    'nn_runs': 100,
    'runs': 2,
    'max_runs': 20,
    'k_folds': 10,
    'verbose': False,
    'layers': [8, 256, 256, 128, 1],
    'epochs': 10_000,
    'validation_step': 100,
    'acc_percentile': 0.1,
    'patience': 3,
    'dropout_rate': 0.2,
    'dropout_uq': 0.5,
    'batch_size': 256,
    'dataset': 'kin8nm',
    'l2_reg': 1e-4,
    'ood_percentile': 90,
    'with_ensembles': True,
    'ensembles_num_models': 5
}

In [ ]:
train_opts = ['patience', 'dropout_rate', 'epochs', 'batch_size', 'validation_step']
config['train_opts'] = {k: config[k] for k in config if k in train_opts}


### Get data

In [ ]:
dataset = build_dataset(config['dataset'], val_split=0.01) 
x_all, y_all = dataset.dataset('train')
x_set, y_set, x_ood, y_ood = split_ood(x_all, y_all, config['ood_percentile'])
config['layers'][0] = x_all.shape[-1]

### Defining support functions


In [ ]:
masks = build_masks(DEFAULT_MASKS)

In [ ]:
emasks = [build_masks(DEFAULT_MASKS) for i in range(config['ensembles_num_models'])]
emasks = {key: [e[key] for e in emasks] for key in masks.keys()}

In [ ]:
def construct_estimator(model, model_type, name):
    if model_type == 'mask': 
        mask = masks[name]
        return build_estimator(
            'mcdue_masked', model, nn_runs=config['nn_runs'], dropout_mask=mask,
            dropout_rate=config['dropout_uq'])
    elif model_type == 'emask': 
        mask = emasks[name]
        return build_estimator(
            'emcdue_masked', model, nn_runs=config['nn_runs'], dropout_mask=mask,
            dropout_rate=config['dropout_uq'])
    else:
        return build_estimator(name, model)


class Evaluator:    
    def __init__(self, x_test, y_test, y_scaler, tag='standard'):
        self.x_test = x_test
        self.y_test = y_test
        self.unscale = lambda y : y_scaler.inverse_transform(y) 
        self.tag = tag
        self.results = []

    def bench(self, model, name, model_type='mask'): 
        predictions = model(self.x_test).cpu().numpy()
        
        errors = np.abs(predictions - self.y_test)
        
        scaled_errors = self.unscale(predictions) - self.unscale(self.y_test)
        rmse = np.sqrt(np.mean(np.square(scaled_errors)))

        estimator = construct_estimator(model, model_type, name)
        if model_type == 'emask':
            name = 'e_' + name
        
        for run in range(config['runs']):
            estimations = estimator.estimate(self.x_test)
            acc, ndcg, ll = get_uq_metrics(estimations, errors, config['acc_percentile'])
            self.results.append([acc, ndcg, ll, rmse, name, self.tag])
            if hasattr(estimator, 'reset'):
                estimator.reset()
            


### Train and generate results

In [ ]:
masks

In [ ]:
mask_results = [] 
kfold_iterator = multiple_kfold(config['k_folds'], len(x_set), config['max_runs'])

for m, (train_idx, val_idx) in enumerate(kfold_iterator):
    x_train, y_train = x_set[train_idx], y_set[train_idx]
    x_val, y_val = x_set[val_idx], y_set[val_idx]
    print("Iteration", m+1)

    x_train, x_val, x_scaler = scale(x_train, x_val)
    y_train, y_val, y_scaler = scale(y_train, y_val)
    x_ood_scaled = x_scaler.transform(x_ood)
    y_ood_scaled = y_scaler.transform(y_ood)
    
    standard_evaluator = Evaluator(x_val, y_val, y_scaler, 'standard')
    ood_evaluator = Evaluator(x_ood_scaled, y_ood_scaled, y_scaler, 'ood')
    
    
    #Dropout 
    optimizer={'type': 'Adadelta', 'weight_decay': config['l2_reg']}
    model = MLP(config['layers'], optimizer=optimizer)
    model.fit((x_train, y_train), (x_val, y_val), **config['train_opts'])
    
    for name in masks: 
        standard_evaluator.bench(model, name, 'mask')
        ood_evaluator.bench(model, name, 'mask')
    
    # RMSE ensemble
    train_opts = config['train_opts'].copy()
    model = MLPEnsemble(config['layers'], n_models=5, reduction='mean')
    model.fit((x_train, y_train), (x_val, y_val), **train_opts)

    standard_evaluator.bench(model, 'eue', 'ensemble')
    ood_evaluator.bench(model, 'eue', 'ensemble')
    
    for name in emasks: 
        standard_evaluator.bench(model, name, 'emask')
        ood_evaluator.bench(model, name, 'emask')
    
    mask_results += standard_evaluator.results
    mask_results += ood_evaluator.results
    


### Plot the results 

In [ ]:
mask_df = pd.DataFrame(mask_results, columns=['Acc', 'NDCG', 'LL', 'RMSE', 'Mask', 'Tag'])

fig = plt.figure(figsize=(16, 32))
plt.suptitle(config['dataset'], fontsize=16)
fig.subplots_adjust(top=0.95)


def boxplot(df, x_label, y_label, i, bottom=0, top=1):
    plt.subplot(8, 1, i)
    plt.xticks(rotation=45)
    if bottom is not None:
        plt.ylim(bottom, top)
    sns.boxplot(data=df, x=x_label, y=y_label, hue='Tag', showfliers=False)

boxplot(mask_df, 'Mask', 'Acc', 1, 0, 1)
boxplot(mask_df, 'Mask', 'NDCG', 2, 0, 1)
boxplot(mask_df[mask_df.Tag=='standard'], 'Mask', 'LL', 3, None)
boxplot(mask_df[mask_df.Tag=='ood'], 'Mask', 'LL', 4, None)
boxplot(mask_df, None, 'RMSE', 5, 0, 1.2*mask_df.RMSE.max())


In [ ]:
timestamp = datetime.now().strftime("%y-%m-%d__%H_%M")
with open(f"data/results_{config['dataset']}_{timestamp}.pkl", 'wb') as f:
    pickle.dump(mask_results, f)
    